In [49]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 
import pandas as pd

df = pd.read_csv('Results/iperf_repeat35.csv')
print(df)


     runtime image_name ressource_limit  repeat  duration type         send  \
0    default      iperf         default       1        10  tcp  944238000.0   
1    default      iperf         default       1        10  udp    1042000.0   
2    default      iperf         default      10        10  tcp  943423000.0   
3    default      iperf         default      10        10  udp    1042000.0   
4    default      iperf         default      11        10  tcp  943313000.0   
..       ...        ...             ...     ...       ...  ...          ...   
835    runsc      iperf             min       7        10  NaN         -1.0   
836    runsc      iperf             min       8        10  tcp  367763000.0   
837    runsc      iperf             min       8        10  NaN         -1.0   
838    runsc      iperf             min       9        10  tcp  350931000.0   
839    runsc      iperf             min       9        10  NaN         -1.0   

        received                                   

In [50]:
filterd = df
# Remove -1 Values
filterd = filterd[(filterd.send != -1)]
print(filterd)

grouped = filterd[(filterd.type == 'tcp')]

print(grouped.to_string())

     runtime image_name ressource_limit  repeat  duration type         send  \
0    default      iperf         default       1        10  tcp  944238000.0   
1    default      iperf         default       1        10  udp    1042000.0   
2    default      iperf         default      10        10  tcp  943423000.0   
3    default      iperf         default      10        10  udp    1042000.0   
4    default      iperf         default      11        10  tcp  943313000.0   
..       ...        ...             ...     ...       ...  ...          ...   
830    runsc      iperf             min       5        10  tcp  333124000.0   
832    runsc      iperf             min       6        10  tcp  327504000.0   
834    runsc      iperf             min       7        10  tcp  337691000.0   
836    runsc      iperf             min       8        10  tcp  367763000.0   
838    runsc      iperf             min       9        10  tcp  350931000.0   

        received                                   

In [51]:
grouped['send_mb'] = 0
grouped['re_mb'] = 0

for index, row in grouped.iterrows():
    grouped.loc[index, 'send_mb'] = row['send']  / 1024 / 1024
    grouped.loc[index, 're_mb'] = row['received'] / 1024 / 1024
    
print(grouped.to_string())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


     runtime image_name ressource_limit  repeat  duration type         send     received                                           filepath     send_mb       re_mb
0    default      iperf         default       1        10  tcp  944238000.0  943841000.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  900.495529  900.116920
2    default      iperf         default      10        10  tcp  943423000.0  943421000.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  899.718285  899.716377
4    default      iperf         default      11        10  tcp  943313000.0  943311000.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  899.613380  899.611473
6    default      iperf         default      12        10  tcp  943678000.0  943545000.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  899.961472  899.834633
8    default      iperf         default      13        10  tcp  943684000.0  943493000.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  899.967194  899.785042
10   default    

In [52]:
# Fehlerrechnung

send_std = grouped.groupby(['runtime', 'ressource_limit'])['send_mb'].std().reset_index(name='send_std')
print(send_std)

re_std = grouped.groupby(['runtime', 'ressource_limit'])['re_mb'].std().reset_index(name='re_std')
print(re_std)

send_mean = grouped.groupby(['runtime', 'ressource_limit'])['send_mb'].mean().reset_index(name='send_mean')
print(send_mean)

re_mean = grouped.groupby(['runtime', 'ressource_limit'])['re_mb'].mean().reset_index(name='re_mean')
print(re_mean)


send_std['send_mean'] = send_mean[['send_mean']]
send_std['re_std'] = re_std[['re_std']]
send_std['send_mean'] = send_mean[['send_mean']]
send_std['re_mean'] = re_mean[['re_mean']]
grouped_std_mean = send_std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['send_abweichung_in_prozent'] = grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['send_mean']
grouped_std_mean['send_konf_intervall_unten'] = grouped_std_mean['send_mean'] - grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909
grouped_std_mean['send_konf_intervall_oben'] = grouped_std_mean['send_mean'] + grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909

grouped_std_mean['re_abweichung_in_prozent'] = grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['re_mean']
grouped_std_mean['re_konf_intervall_unten'] = grouped_std_mean['re_mean'] - grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909
grouped_std_mean['re_konf_intervall_oben'] = grouped_std_mean['re_mean'] + grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909

grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

    runtime ressource_limit   send_std
0   default         default   0.337549
1   default             max   0.386225
2   default             min   0.310262
3      kata         default   1.250538
4      kata             max   1.413461
5      kata             min   0.891736
6    katafc         default   4.828796
7    katafc             max   4.708586
8    katafc             min   3.303640
9     runsc         default   0.657144
10    runsc             max   1.408692
11    runsc             min  50.574158
    runtime ressource_limit     re_std
0   default         default   0.316005
1   default             max   0.185522
2   default             min   0.278502
3      kata         default   1.777234
4      kata             max   1.908429
5      kata             min   1.293690
6    katafc         default   5.397605
7    katafc             max   5.196790
8    katafc             min   3.723974
9     runsc         default   1.127308
10    runsc             max   1.474681
11    runsc             m

In [37]:

######
# Leistung indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

color_dict = {'default': 0, 'kata': 2, 'katafc': 3, 'runsc': 4, 'runnc': 5} 
legend_dict = {'default': 'Docker (runc)', 'kata': 'Kata', 'katafc': 'Kata FC', 'runsc': 'gVisor (runsc)', 'runnc': 'Nabla (runnc)'} 
dash_dict = {'default': 'solid', 'max': 'dashed', 'min': 'dotted'}

p = figure(x_axis_type='linear', width=1500)
colour_counter = 0
for runtime in runtimes:
    filterd_rnt=filterd[filterd['runtime'] == runtime]

    source = ColumnDataSource(pd.DataFrame(filterd_rnt))
    #print(source.data)

    ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
    #print(ressource_limits)

    for limit in ressource_limits:
        filterd_limit=filterd_rnt[filterd_rnt['ressource_limit'] == limit]

        source = ColumnDataSource(pd.DataFrame(filterd_limit))
        #print(source.data)
        #glyph = VArea(x='number_of_targets', y1='min_requests_per_second', y2='max_requests_per_second', fill_color=bokeh.palettes.d3['Category20'][12][1], fill_alpha=0.3)
        #p.add_glyph(default_source, glyph)
        p.line(x='number_of_targets', y='send_mean', line_width=2, line_dash=dash_dict[limit], source=source, color=bokeh.palettes.d3['Category10'][6][color_dict[runtime]], legend_label=legend_dict[runtime] + ' - ' + limit)


        p.add_layout(
          Whisker(source=source, base='send_mean', upper="send_konf_intervall_oben", lower="send_konf_intervall_unten", level="overlay")
        )

    colour_counter = colour_counter + 1

p.title.text ='Arbeitsspeicherverbrauch nach Anzahl der parellen Container für ' + image_name
p.xaxis.axis_label = 'Anzahl an parallelen Containern'
p.yaxis.axis_label = 'Arbeitsspeicherverbrauch [MB]'
p.x_range.start = 1
p.xaxis.ticker = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
p.legend.click_policy="hide"
p.legend.location = "top_left"
p.legend.background_fill_alpha = 0.5

output_file("requests_pro_sec_anzahl_container_" + image_name + ".html")
output_notebook()
show(p)

NameError: name 'ass' is not defined

In [65]:
######
# Leistung indivduell
####
source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

runtimes = list(dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list(dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

for limit in ressource_limits:
    filterd_lmt=grouped_std_mean[grouped_std_mean['ressource_limit'] == limit]
    source = ColumnDataSource(pd.DataFrame(filterd_lmt))
    #source = ColumnDataSource(data=dict(x=x, mean=source.data['send_mean'].tolist(), oben=source.data['send_konf_intervall_oben'].tolist(), unten=source.data['send_konf_intervall_unten'].tolist()))
    print(source.data)
    
    runtime = source.data['runtime'].tolist()
          
    p = figure(x_range=runtime, width=1000)

    # use the palette to colormap based on the the x[1:2] values
    #fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

    #p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)
    p.vbar(x='runtime', top='send_mean', source=source, width=0.70)

    p.add_layout(
      Whisker(source=source, base='runtime', upper="send_konf_intervall_oben", lower="send_konf_intervall_unten", level="overlay")
    )


    p.title.text ='Ram Verbrauch für Image ' +  ' mit der Skalierung' 
    p.xaxis.axis_label = 'Ressourcelimit & Runtime'
    p.yaxis.axis_label = 'Ram Verbrauch'

    p.xaxis.axis_label_text_font_size = '11pt'
    p.yaxis.axis_label_text_font_size = '11pt'



    output_notebook()

    show(p)

{'index': array([0, 3, 6]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['default', 'default', 'default'], dtype=object), 'send_std': array([0., 0., 0.]), 'send_mean': array([0.99, 0.99, 0.99]), 'send_abweichung_in_prozent': array([0., 0., 0.]), 'send_konf_intervall_unten': array([0.99, 0.99, 0.99]), 'send_konf_intervall_oben': array([0.99, 0.99, 0.99]), 'send_docker_abweichung': array([100., 100., 100.]), 'typ': array(['udp', 'udp', 'udp'], dtype=object)}


Loading BokehJS ...

{'index': array([1, 4, 7]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['max', 'max', 'max'], dtype=object), 'send_std': array([0., 0., 0.]), 'send_mean': array([0.99, 0.99, 0.99]), 'send_abweichung_in_prozent': array([0., 0., 0.]), 'send_konf_intervall_unten': array([0.99, 0.99, 0.99]), 'send_konf_intervall_oben': array([0.99, 0.99, 0.99]), 'send_docker_abweichung': array([100., 100., 100.]), 'typ': array(['udp', 'udp', 'udp'], dtype=object)}


Loading BokehJS ...

{'index': array([2, 5, 8]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['min', 'min', 'min'], dtype=object), 'send_std': array([0., 0., 0.]), 'send_mean': array([0.99, 0.99, 0.99]), 'send_abweichung_in_prozent': array([0., 0., 0.]), 'send_konf_intervall_unten': array([0.99, 0.99, 0.99]), 'send_konf_intervall_oben': array([0.99, 0.99, 0.99]), 'send_docker_abweichung': array([100., 100., 100.]), 'typ': array(['udp', 'udp', 'udp'], dtype=object)}


Loading BokehJS ...

In [53]:
print(grouped_std_mean.to_string())


    runtime ressource_limit  send_std  send_mean  re_std  re_mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben  re_abweichung_in_prozent  re_konf_intervall_unten  re_konf_intervall_oben
0   default         default      0.34     900.07    0.32   899.82                        0.01                     899.98                    900.17                      0.01                   899.73                  899.91
1   default             max      0.39     900.09    0.19   899.85                        0.01                     899.98                    900.20                      0.01                   899.80                  899.91
2   default             min      0.31     900.00    0.28   899.81                        0.01                     899.91                    900.09                      0.01                   899.73                  899.89
3      kata         default      1.25     899.00    1.78   898.42                        0.04                   

In [54]:
grouped_std_mean['send_docker_abweichung'] = 0
grouped_std_mean['re_docker_abweichung'] = 0
grouped_std_mean['typ'] = 'tcp'

#print(grouped_std_mean.loc[ 273 , : ])

for index, row in grouped_std_mean.iterrows():
    send_docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['send_mean']].reset_index().get_value(0, 'send_mean')
    re_docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['re_mean']].reset_index().get_value(0, 're_mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'send_docker_abweichung'] = row['send_mean'] * 100 / send_docker_mean
    grouped_std_mean.loc[index, 're_docker_abweichung'] = row['re_mean'] * 100 / re_docker_mean

grouped_std_mean_tcp = grouped_std_mean.round(2)

print(grouped_std_mean_tcp.to_string())


    runtime ressource_limit  send_std  send_mean  re_std  re_mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben  re_abweichung_in_prozent  re_konf_intervall_unten  re_konf_intervall_oben  send_docker_abweichung  re_docker_abweichung  typ
0   default         default      0.34     900.07    0.32   899.82                        0.01                     899.98                    900.17                      0.01                   899.73                  899.91                  100.00                100.00  tcp
1   default             max      0.39     900.09    0.19   899.85                        0.01                     899.98                    900.20                      0.01                   899.80                  899.91                  100.00                100.00  tcp
2   default             min      0.31     900.00    0.28   899.81                        0.01                     899.91                    900.09                      0.01         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [55]:
filterd = df
# Remove -1 Values
filterd = filterd[(filterd.send != -1)]
print(filterd)

grouped = filterd[(filterd.type == 'udp')]

print(grouped.to_string())

     runtime image_name ressource_limit  repeat  duration type         send  \
0    default      iperf         default       1        10  tcp  944238000.0   
1    default      iperf         default       1        10  udp    1042000.0   
2    default      iperf         default      10        10  tcp  943423000.0   
3    default      iperf         default      10        10  udp    1042000.0   
4    default      iperf         default      11        10  tcp  943313000.0   
..       ...        ...             ...     ...       ...  ...          ...   
830    runsc      iperf             min       5        10  tcp  333124000.0   
832    runsc      iperf             min       6        10  tcp  327504000.0   
834    runsc      iperf             min       7        10  tcp  337691000.0   
836    runsc      iperf             min       8        10  tcp  367763000.0   
838    runsc      iperf             min       9        10  tcp  350931000.0   

        received                                   

In [56]:
grouped['send_mb'] = 0

for index, row in grouped.iterrows():
    grouped.loc[index, 'send_mb'] = row['send']  / 1024 / 1024
    
print(grouped.to_string())

     runtime image_name ressource_limit  repeat  duration type       send  received                                           filepath   send_mb
1    default      iperf         default       1        10  udp  1042000.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  0.993729
3    default      iperf         default      10        10  udp  1042000.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  0.993729
5    default      iperf         default      11        10  udp  1041990.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  0.993719
7    default      iperf         default      12        10  udp  1042000.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  0.993729
9    default      iperf         default      13        10  udp  1042000.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defau...  0.993729
11   default      iperf         default      14        10  udp  1042000.0      -1.0  /opt/auswertung/Raw/bench_iperf_repeat35/defa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
# Fehlerrechnung

send_std = grouped.groupby(['runtime', 'ressource_limit'])['send_mb'].std().reset_index(name='send_std')
print(send_std)

send_mean = grouped.groupby(['runtime', 'ressource_limit'])['send_mb'].mean().reset_index(name='send_mean')
print(send_mean)

send_std['send_mean'] = send_mean[['send_mean']]
grouped_std_mean = send_std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['send_abweichung_in_prozent'] = grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['send_mean']
grouped_std_mean['send_konf_intervall_unten'] = grouped_std_mean['send_mean'] - grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909
grouped_std_mean['send_konf_intervall_oben'] = grouped_std_mean['send_mean'] + grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909

grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

   runtime ressource_limit  send_std
0  default         default  0.000005
1  default             max  0.000004
2  default             min  0.000004
3     kata         default  0.000004
4     kata             max  0.000004
5     kata             min  0.000004
6   katafc         default  0.000004
7   katafc             max  0.000004
8   katafc             min  0.000004
   runtime ressource_limit  send_mean
0  default         default   0.993725
1  default             max   0.993727
2  default             min   0.993728
3     kata         default   0.993726
4     kata             max   0.993726
5     kata             min   0.993726
6   katafc         default   0.993727
7   katafc             max   0.993727
8   katafc             min   0.993726
   runtime ressource_limit  send_std  send_mean
0  default         default  0.000005   0.993725
1  default             max  0.000004   0.993727
2  default             min  0.000004   0.993728
3     kata         default  0.000004   0.993726
4     kata

In [62]:
grouped_std_mean['send_docker_abweichung'] = 0
grouped_std_mean['typ'] = 'udp'

#print(grouped_std_mean.loc[ 273 , : ])

for index, row in grouped_std_mean.iterrows():
    send_docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['send_mean']].reset_index().get_value(0, 'send_mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'send_docker_abweichung'] = row['send_mean'] * 100 / send_docker_mean

grouped_std_mean_udp = grouped_std_mean.round(2)

print(grouped_std_mean_udp.to_string())

   runtime ressource_limit  send_std  send_mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben  send_docker_abweichung  typ
0  default         default       0.0       0.99                         0.0                       0.99                      0.99                   100.0  udp
1  default             max       0.0       0.99                         0.0                       0.99                      0.99                   100.0  udp
2  default             min       0.0       0.99                         0.0                       0.99                      0.99                   100.0  udp
3     kata         default       0.0       0.99                         0.0                       0.99                      0.99                   100.0  udp
4     kata             max       0.0       0.99                         0.0                       0.99                      0.99                   100.0  udp
5     kata             min       0.0       0.99     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


In [64]:
result = pd.concat([grouped_std_mean_udp, grouped_std_mean_tcp], sort=False)


print(result.to_string())

export_csv = result.to_csv (r'Results/iperf_Gewichtung_Docker.csv', index = None, header=True, decimal=',')

    runtime ressource_limit  send_std  send_mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben  send_docker_abweichung  typ  re_std  re_mean  re_abweichung_in_prozent  re_konf_intervall_unten  re_konf_intervall_oben  re_docker_abweichung
0   default         default      0.00       0.99                        0.00                       0.99                      0.99                  100.00  udp     NaN      NaN                       NaN                      NaN                     NaN                   NaN
1   default             max      0.00       0.99                        0.00                       0.99                      0.99                  100.00  udp     NaN      NaN                       NaN                      NaN                     NaN                   NaN
2   default             min      0.00       0.99                        0.00                       0.99                      0.99                  100.00  udp     NaN      NaN      